<a href="https://colab.research.google.com/github/FastBackendML/FastBackendML/blob/main/Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.10 Использование CPU и GPU на примере простой НС

In [1]:
import torch

In [4]:
torch.cuda.is_available()

False

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
t1 = torch.arange(1,6, dtype=torch.int32, device=device)
t2 = torch.ones(15, dtype=torch.float32, device=device).view(3,5)
t_res = t2.matmul(t1.float()).to(device='cpu')

In [24]:
t_res

tensor([15., 15., 15.])

Task 2

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [26]:
targets = torch.rand(1000).to(device)
predict = torch.rand(1000).to(device)

In [27]:
Q = torch.sum(torch.pow((targets - predict), 2)) / 1000

In [29]:
Q = Q.to('cpu')

In [30]:
Q

tensor(169.7121)

Task 3

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

w = torch.rand(1, 16) # вектор-строка
X = torch.rand(16, 16) # матрица 16 x 16
I = torch.eye(16) # единичная матрица 16 x 16
lm = 0.5 # значение лямбда